# Goal Oriented Generation 

Based on prompts from [Creating Suspenseful Stories with Large Language Models](https://arxiv.org/abs/2402.17119).

In [55]:
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# See langchain_aws for more information on other bedrock models
# This is the model we will be using for this example

model = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0.7},
)

# Setting a role for the chat model to act as a short story writer

role = """You are a short story writer with a passion for creative writing and storytelling. 
Your task is to create engaging stories that captivate your audience and leave them wanting more. 
"""

def prompt_from_template(template: str):
    return ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", template)
    ])


# Background Setup

This is the world knowledge needed to tell a coherent story about the protagonist.  We'll need the following items:

- Protagonist's name and occupution
- The goal they want to achieve
- The consequence for not achieving the goal

From those items, we'll prompt the model to write an introductory paragraph


In [56]:
genre = "Espionage Thriller"
name_pormpt = prompt_from_template(template="The name and the occupation of a protagonist of a {genre}:")

name_chain = name_pormpt | model | StrOutputParser()
name_occupation = name_chain.invoke({"genre":genre})

jinja_vars = {
    "genre": genre,
    "name_occupation": name_occupation
}

In [57]:
goal_instructions = """
<protagonist_background>
<name_occupation>{name_occupation}</name_occupation>
</protagonist_background>

A very concrete goal the protagonist wants to achieve:"""

goal_prompt = prompt_from_template(template=goal_instructions)

gaol_chain = goal_prompt | model | StrOutputParser()
goal = gaol_chain.invoke({"name_occupation":name_occupation})

jinja_vars["goal"] = goal

In [58]:

situtaion = """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
</protagonist_background>

Tell me about a dire situation that the protagonist would be put in, if the protagonist couldn’t achieve the goal:
"""

situtaion_prompt = prompt_from_template(template=situtaion)

situation_chain = situtaion_prompt | model | StrOutputParser()
dire_situation = situation_chain.invoke({"name_occupation":name_occupation, "goal":goal})

jinja_vars["dire_situation"] = dire_situation

In [59]:
intro_instruction =  """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</protagonist_background>

Write an introductory paragraph of a story that covers all the above information:
"""

intro_prompt = prompt_from_template(template=intro_instruction)
intro_chain = intro_prompt | model | StrOutputParser()
intro = intro_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})

jinja_vars["intro"] = intro

Here is an introductory paragraph for the espionage thriller story featuring the protagonist Evelyn Harrington:

Evelyn Harrington's footsteps echoed through the dimly lit corridor as she made her way towards the secure briefing room. As a highly trained covert intelligence operative, she was no stranger to the shadows of international espionage, but the weight of her current mission pressed heavily on her mind. Intelligence had uncovered a dangerous conspiracy, one that threatened to unravel the delicate balance of global power and plunge the world into chaos. Evelyn knew she was the only one who could infiltrate the criminal organization at the heart of this plot, gathering the crucial evidence needed to expose their nefarious schemes. Failure was not an option - the consequences were unthinkable. With a steely determination in her eyes, Evelyn pushed open the door, ready to confront the gravity of the task that lay ahead.


# Outline  Planning


In [60]:
action_1_instructions =  """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</protagonist_background>

In a narrative style, write a paragraph about concrete action the protagonist is most likely to take, 
in order to achieve the goal:\n
"""

action_1_prompt = prompt_from_template(template=action_1_instructions)
action_1_chain = action_1_prompt | model | StrOutputParser()
action_1 = action_1_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})

jinja_vars["action_1"] = action_1

<action>
Determined to uncover and dismantle the dangerous international conspiracy, Evelyn Harrington immediately springs into action, drawing upon her extensive training and experience as a covert intelligence operative. First and foremost, she meticulously analyzes the limited intelligence she has gathered so far, searching for any potential leads or weaknesses in the criminal organization's operations.

With a keen eye for detail and an analytical mind honed by years of field work, Evelyn begins to piece together the intricate web of connections and resources that the syndicate has amassed. She identifies key figures within the organization, mapping out their roles, relationships, and potential vulnerabilities. Armed with this information, Evelyn devises a comprehensive strategy, one that will allow her to infiltrate the criminal network and gather the evidence needed to expose their nefarious plans.

Evelyn knows that time is of the essence, as the conspiracy continues to unfold, 

In [61]:
action_1_reason_inst = """The protagonist tries to perform the following action: {action_1}. 
But the protagonist still fails to achieve the goal for a reason. 
Write a paragraph about the reason for the failure in a narrative style:\n"""

action_1_reason_prompt = prompt_from_template(template=action_1_reason_inst)
action_1_reason_chain = action_1_reason_prompt | model | StrOutputParser()
action_1_reason = action_1_reason_chain.invoke({"action_1":action_1})

jinja_vars["action_1_reason"] = action_1_reason

Evelyn's heart raced as she executed her meticulously planned infiltration of the criminal syndicate. She had gathered the intelligence, mapped out the connections, and assembled her team of trusted operatives. Yet, as she made her way deeper into the organization's stronghold, a nagging sense of unease crept up her spine.

It was the little things that began to unravel her carefully constructed strategy. A guard posted in an unexpected location, a security protocol that had been updated since her last intel briefing, a subtle shift in the syndicate's operations that threw her timing off. Evelyn found herself constantly adapting, improvising, and reacting to the unpredictable nature of her adversaries.

Despite her extensive training and experience, Evelyn realized that she had underestimated the syndicate's adaptability and resilience. They were not a static target, but a constantly evolving entity, able to anticipate and counter her moves with unsettling precision. As she pushed forw

In [62]:
from jinja2 import Environment, select_autoescape, FileSystemLoader
env = Environment(
    loader=FileSystemLoader("."),
    autoescape=select_autoescape()
)

template = env.get_template("goal_template.jinja")

story = template.render(**jinja_vars)

with open("story.md", "w") as f:
    f.write(story)

# Backgroud Infommation

## Genre 

Espionage Thriller

## Name & Occupation

Here's a protagonist for an espionage thriller:

Name: Evelyn Harrington
Occupation: Covert Intelligence Operative

Evelyn Harrington is a highly trained and experienced covert intelligence operative working for a secretive government agency. She is an expert in counter-espionage, surveillance, and information gathering, with a sharp mind and an unwavering sense of duty. 

Evelyn's job often takes her into the shadows, as she navigates the dangerous world of international espionage, uncovering plots and thwarting the efforts of foreign adversaries. She is adept at blending into her surroundings, adopting various personas, and using her keen observational skills to stay one step ahead of her targets.

Despite the high-stakes nature of her work, Evelyn maintains a cool, collected demeanor, driven by an unwavering moral compass and a determination to protect her country's interests. However, her personal life is